# Importando as bibliotecas

In [2]:
import pandas as pd
import numpy as np 
import plotly.express as px
from pathlib import Path
import inflection as inflection
import folium
from folium.plugins import MarkerCluster


# Funções que serão utilizadas no código

In [3]:
## função para descobrir quais são as colunas que tem valores iguais 

def coluna_values_unicos (df): 

    #extraindo as colunas do dataframe
    colunas = list(df.columns)

    colunas_unicas = [coluna_unica for coluna_unica in colunas if len(df[coluna_unica].unique()) == 1]

    return colunas_unicas

## função para preencher os países: 

COUNTRIES = {
   1: "India",
   14: "Australia",
   30: "Brazil",
   37: "Canada",
   94: "Indonesia",
   148: "New Zeland",
   162: "Philippines",
   166: "Qatar",
   184: "Singapure",
   189: "South Africa",
   191: "Sri Lanka",
   208: "Turkey",
   214: "United Arab Emirates",
   215: "England",
   216: "United States of America",
}
def country_name(country_id): return COUNTRIES[country_id]

## função para criação do nome das cores 

COLORS = {
   "3F7E00": "darkgreen",
   "5BA829": "green",
   "9ACD32": "lightgreen",
   "CDD614": "orange",
   "FFBA00": "red",
   "CBCBC8": "darkred",
   "FF7800": "darkred",
}
def color_name(color_code): return COLORS[color_code]

## função para renomear os nomes dos dataframes: 

def rename_columns(dataframe):
   df = dataframe.copy()
   title = lambda x: inflection.titleize(x) 
   snakecase = lambda x: inflection.underscore(x) 
   spaces = lambda x: x.replace(" ", "")
   cols_old = list(df.columns)
   cols_old = list(map(title, cols_old)) 
   cols_old = list(map(spaces, cols_old)) 
   cols_new = list(map(snakecase, cols_old)) 
   df.columns = cols_new
   return df

## função para categorizar a comida 

def create_price_tye(price_range):
   if price_range == 1: 
      return "cheap"
   elif price_range == 2: 
      return "normal"
   elif price_range == 3: 
      return "expensive"
   else:
      return "gourmet" 

# Abrindo o arquivo e trazendo algumas infos do dataframe

In [4]:
## caminho do arquivo: 

arquivo = Path.cwd()/'dataset'/'zomato.csv'

## criando um dataframe

df = pd.read_csv(arquivo)

In [5]:
## setando para ver todas as configurações: 

pd.set_option('display.max_columns',None)

In [6]:
## investigando o dataframe

df.info()

## criando coluna de estatística descritiva 

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7527 entries, 0 to 7526
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         7527 non-null   int64  
 1   Restaurant Name       7527 non-null   object 
 2   Country Code          7527 non-null   int64  
 3   City                  7527 non-null   object 
 4   Address               7527 non-null   object 
 5   Locality              7527 non-null   object 
 6   Locality Verbose      7527 non-null   object 
 7   Longitude             7527 non-null   float64
 8   Latitude              7527 non-null   float64
 9   Cuisines              7512 non-null   object 
 10  Average Cost for two  7527 non-null   int64  
 11  Currency              7527 non-null   object 
 12  Has Table booking     7527 non-null   int64  
 13  Has Online delivery   7527 non-null   int64  
 14  Is delivering now     7527 non-null   int64  
 15  Switch to order menu 

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Votes
count,7.527000e+03,7527.000000,7527.000000,7527.000000,7.527000e+03,7527.000000,7527.000000,7527.000000,7527.0,7527.000000,7527.000000,7527.000000
mean,1.055689e+07,93.065365,33.764092,19.332787,7.152113e+03,0.062176,0.352464,0.174306,0.0,2.624552,4.118055,616.401488
std,7.075141e+06,99.031229,77.395241,23.255979,2.906068e+05,0.241491,0.477769,0.379397,0.0,0.959794,0.680191,1127.697932
min,5.490000e+02,1.000000,-122.700455,-41.330428,0.000000e+00,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000
25%,3.500060e+06,1.000000,-4.258142,12.923378,6.000000e+01,0.000000,0.000000,0.000000,0.0,2.000000,4.000000,152.000000
50%,7.701457e+06,30.000000,73.785121,25.246955,2.900000e+02,0.000000,0.000000,0.000000,0.0,3.000000,4.200000,335.000000
75%,1.714715e+07,214.000000,79.833706,31.636552,6.000000e+02,0.000000,1.000000,0.000000,0.0,3.000000,4.500000,663.000000
max,1.904028e+07,216.000000,175.310552,55.976980,2.500002e+07,1.000000,1.000000,1.000000,0.0,4.000000,4.900000,41333.000000


# Limpeza de Dados

In [7]:
## pelo método info conseguimos identificar que tem 15 linhas na coluna 'Cusines' que são nulas. Desse modo vamos remover essas linhas 

df = df.dropna().reset_index(drop=True)

## removendo colunas com todos os valores iguais 

df = df.drop(coluna_values_unicos(df),axis=1)

## colocando o nome dos países pela função e criando uma nova coluna

df['Country Name'] = df['Country Code'].apply(country_name)

## criando uma coluna com o nome das cores

df['Color Name'] = df['Rating color'].apply(color_name)

## removendo linhas duplicadas

df = df.drop_duplicates().reset_index(drop=True)

## renomeando o nome das colunas 

df = rename_columns(df)

## categoriazando as cozinhas por apenas um tipo 

df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

## usando a função de categorização para comida

df['price_range_name'] = df['price_range'].apply(create_price_tye)

# Perguntas Geral

In [8]:
# 1 Quantos restaurantes únicos estão registrados 

restaurantes_unicos = len(df['restaurant_id'].unique())

restaurantes_unicos

6929

In [9]:
## 2 Quantos países únicos estão registrados

paises_unicos = len(df['country_name'].unique())

paises_unicos

15

In [10]:
## 3 Quantas cidades únicas estão registradas 

cidades_unicas = len(df['city'].unique())

cidades_unicas

125

In [11]:
## 4 Qual o total de avaliações 

total_avaliacoes = df['votes'].sum()

total_avaliacoes

4194533

In [12]:
## 5 Qual o total de tipos de culinárias registradas 

tipos_culinarias = df['cuisines'].nunique()

tipos_culinarias

165

# País

### 1. Qual o nome do país que possui mais cidades registradas

In [13]:
df.loc[:,['country_name','city']].groupby(['country_name']).nunique().sort_values('city',ascending=False).reset_index().iloc[0,0]

'India'

### 2. Qual o nome do país que possui mais restaurantes registrados

In [14]:
df.loc[:,['country_name','restaurant_id']].groupby(['country_name']).nunique().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'India'

### 3. Qual o nome do país que possui mais restaurantes com nível de preço igual a 4 registrados? 

In [15]:
df.loc[df['price_range'] >= 4,['country_name','restaurant_id']].groupby(['country_name']).nunique().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'United States of America'

### 4. Qual o nome do país que possui a maior quantidade de tipos de culinárias distintos

In [16]:
df.loc[:,['country_name','cuisines']].groupby(['country_name']).nunique().sort_values('cuisines',ascending=False).reset_index().iloc[0,0] 

'India'

### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas

In [17]:
df.loc[:,['country_name','votes']].groupby(['country_name']).sum().sort_values('votes',ascending=False).reset_index().iloc[0,0]

'India'

### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entregas

In [18]:
df.loc[:,['country_name','is_delivering_now']].groupby('country_name').sum().sort_values('is_delivering_now',ascending=False).reset_index().iloc[0,0]

'India'

### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas

In [19]:
df.loc[(df['has_table_booking'] == 1),['country_name','restaurant_id']].groupby('country_name').count().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'India'

### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registradas

In [20]:
df.loc[:,['country_name','votes']].groupby(['country_name']).mean().sort_values('votes',ascending=False).reset_index().iloc[0,0]

'Indonesia'

### 9. Qual o nome do país que possui, na média, a maior nota média registrada

In [21]:
df.loc[:,['country_name','aggregate_rating']].groupby(['country_name']).mean().sort_values('aggregate_rating',ascending=False).reset_index().iloc[0,0]

'Indonesia'

### 10. Qual o nome do país que possui, na média, a menor nota média registrada

In [22]:
df.loc[:,['country_name','aggregate_rating']].groupby(['country_name']).mean().sort_values('aggregate_rating',ascending=False).reset_index().iloc[-1,0]

'Brazil'

### 11. Qual a média de preço de um prato para dois por país?

In [23]:
df.loc[:,['country_name','average_cost_for_two']].groupby('country_name').mean().reset_index()

,country_name,average_cost_for_two
0,Australia,138959.783333
1,Brazil,138.812500
2,Canada,41.861111
3,England,43.510000
4,India,704.400514
5,Indonesia,303000.000000
6,New Zeland,62.154812
7,Philippines,1227.825000
8,Qatar,174.000000
9,Singapure,141.437500


# Cidade

### 1. Qual o nome da cidade que possui mais restaurantes registrados

In [24]:
df.loc[:,['city','restaurant_id']].groupby('city').nunique().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'Abu Dhabi'

### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [25]:
df.loc[(df['aggregate_rating'] > 4),['city','restaurant_id']].groupby('city').count().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'Bangalore'

### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [26]:
df.loc[(df['aggregate_rating']<= 2.5),['city','restaurant_id']].groupby('city').count().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'Gangtok'

### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois

In [27]:
df.loc[:,['average_cost_for_two','city']].groupby('city').mean().sort_values('average_cost_for_two',ascending=False).reset_index().iloc[0,0]

'Adelaide'

### 5. Qual o nome da cidade que possuir a maior quantidade de tipos de culinária distintas?

In [28]:
df.loc[:,['city','cuisines']].groupby('city').nunique().sort_values('cuisines',ascending=False).reset_index().iloc[0,0]

'Birmingham'

### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas? 

In [29]:
df.loc[(df['has_table_booking'] == 1),['city','restaurant_id']].groupby('city').count().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'Bangalore'

### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [30]:
df.loc[(df['is_delivering_now'] == 1),['city','restaurant_id']].groupby('city').count().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'Vadodara'

### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [31]:
df.loc[(df['has_online_delivery'] == 1),['city','restaurant_id']].groupby('city').count().sort_values('restaurant_id',ascending=False).reset_index().iloc[0,0]

'Bhopal'

### Top 10 cidades com mais restaurantes com tipos de culinários distintos

In [32]:
df_aux = df.loc[:,['city','cuisines','country_name']].groupby(['city','country_name']).nunique().sort_values('cuisines',ascending=False).reset_index().head(10)

# Restaurantes

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [33]:
df.loc[df['votes'].idxmax(),'restaurant_name']

'Bawarchi'

### 2. Qual o nome do restaurante com a maior nota média?

In [34]:
df.loc[:,['restaurant_id','restaurant_name','country_name','city','cuisines','aggregate_rating','votes']].sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True]).iloc[0,1]

'Indian Grill Room'

### 3. Qual o nome do restaurante que possui o maior valor de um prato para duas pessoas? 

In [35]:
df.loc[:,['restaurant_id','restaurant_name','average_cost_for_two']].sort_values(['average_cost_for_two','restaurant_id'],ascending=[False,True]).iloc[0,1]

"d'Arry's Verandah Restaurant"

### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação

In [36]:
df.loc[(df['cuisines'] == 'Brazilian'),['restaurant_id','restaurant_name','aggregate_rating']].sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True]).iloc[0,1]

'Loca Como tu Madre'

### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação

In [37]:
df.loc[(df['cuisines'] == 'Brazilian') & (df['country_name'] == 'Brazil'),['restaurant_id','restaurant_name','aggregate_rating']].sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True]).iloc[0,1]

'Braseiro da Gávea'

### 6. Os restaurantes que aceitam pedido online, são também, na média, os restaurantes que mais possuem avaliações registradas

In [38]:
df.loc[:,['has_online_delivery','votes']].groupby('has_online_delivery').mean().sort_values('votes',ascending=False).reset_index()

,has_online_delivery,votes
0,1,838.821664
1,0,479.421018


### 7. Os restaurantes que fazerm reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [39]:
df.loc[:,['has_table_booking','average_cost_for_two']].groupby('has_table_booking').mean().sort_values('average_cost_for_two',ascending=False).reset_index()

,has_table_booking,average_cost_for_two
0,1,69998.423810
1,0,3488.596866


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [40]:
(df.loc[((df['cuisines']=='BBQ') | (df['cuisines'] == 'Japanese')) & (df['country_name'] == 'United States of America'),['cuisines','average_cost_for_two']]
   .groupby('cuisines').mean().reset_index().sort_values('average_cost_for_two',ascending=False))

,cuisines,average_cost_for_two
1,Japanese,56.406250
0,BBQ,39.642857


# Tipos de Culinária

In [41]:
def tipos_de_culinarias(df,cozinha,classification=True): 

    df1 = (df.loc[(df['cuisines']==cozinha),['restaurant_id','restaurant_name','aggregate_rating']]
                         .sort_values(['aggregate_rating','restaurant_id'],ascending=[classification,True]))
    restaurant_name = df1.iloc[0,1]

    id_rest = df1.index[0]

    aggregate_rating = df.loc[(df.index==id_rest),['aggregate_rating']].iloc[0,0]

    pais = df.loc[(df.index==id_rest),['country_name']].iloc[0,0]

    average_cost_for_two = df.loc[(df.index==id_rest),['average_cost_for_two']].iloc[0,0]

    currency = df.loc[(df.index==id_rest),['currency']].iloc[0,0]

    city = df.loc[(df.index==id_rest),['city']].iloc[0,0]

    
    return restaurant_name, aggregate_rating, pais, average_cost_for_two, currency,city


### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com maior média de avaliação?

In [42]:
df.loc[(df['cuisines']=='Italian'),['restaurant_id','restaurant_name','aggregate_rating']].sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True]).iloc[0,1]

'Darshan'

In [43]:
tipos_de_culinarias(df,'Italian',False)[0]

'Darshan'

### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [44]:
df.loc[(df['cuisines']=='Italian'),['restaurant_id','restaurant_name','aggregate_rating']].sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True]).iloc[0,1]

'Avenida Paulista'

### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [45]:
df.loc[(df['cuisines']=='American'),['restaurant_id','restaurant_name','aggregate_rating']].sort_values(['aggregate_rating','restaurant_id'],ascending=[False,True]).iloc[0,1]

'Burger & Lobster'

### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [46]:
df.loc[(df['cuisines']=='American'),['restaurant_id','restaurant_name','aggregate_rating']].sort_values(['aggregate_rating','restaurant_id'],ascending=[True,True]).iloc[0,1]

'Alston Bar & Beef'

### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [47]:
tipos_de_culinarias(df,'Arabian',False)

('Mandi@36', 4.7, 'India', 600, 'Indian Rupees(Rs.)', 'Hyderabad')

### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [48]:
tipos_de_culinarias(df,'Arabian',True)

('Raful', 0.0, 'Brazil', 120, 'Brazilian Real(R$)', 'São Paulo')

### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [49]:
tipos_de_culinarias(df,'Japanese',False)

('Sushi Samba', 4.9, 'England', 110, 'Pounds(£)', 'London')

### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [50]:
tipos_de_culinarias(df,'Japanese',True)

('Banzai Sushi', 0.0, 'Brazil', 90, 'Brazilian Real(R$)', 'Brasília')

### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [51]:
df.loc[df['restaurant_name'] == 'Braseiro da Gávea',:]

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,country_name,color_name,price_range_name
191,7300955,Braseiro da Gávea,30,Rio de Janeiro,"Praça Santos Dumont, 116, Gávea, Rio de Janeiro",Gávea,"Gávea, Rio de Janeiro",-43.227042,-22.973507,Brazilian,100,Brazilian Real(R$),0,0,0,4,4.9,3F7E00,Excellent,47,Brazil,darkgreen,gourmet


### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [52]:
tipos_de_culinarias(df,'Home-made',True)

('GurMekan Restaurant', 3.7, 'Turkey', 30, 'Turkish Lira(TL)', 'Ankara')

### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [53]:
df.loc[:,['cuisines','average_cost_for_two']].groupby('cuisines').max().sort_values('average_cost_for_two',ascending=False).reset_index().head(1)

,cuisines,average_cost_for_two
0,Modern Australian,25000017


### 12. Qual o tipo de culinária que possui a maior nota média?

In [54]:
(df.loc[:,['cuisines','aggregate_rating','votes']]
   .groupby('cuisines').agg({'aggregate_rating':'max','votes':'sum'})
   .sort_values(['aggregate_rating','votes'],ascending=[False,False])
   .reset_index()
   .iloc[0,0])

'North Indian'

### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [55]:
df.loc[(df['has_online_delivery'] == 1) & (df['is_delivering_now'] == 1),['cuisines']].value_counts().reset_index().head(1)

,cuisines,count
0,North Indian,317
